In [ ]:
import RNA
import infrared as ir
import infrared.rna as rna
import random
import math
from collections import Counter
import time
import argparse


In [ ]:
parser = argparse.ArgumentParser(description='A program for creating new structures from a fasta file using Infrared.')
parser.add_argument('-i', '--input', type=str, help='the input fasta file')

In [ ]:
file = "ArchiveII.fa"
structures = []
sequences = []
with open(file, "r") as f:
    for i, line in enumerate(f):
        if i%3==2:
            structures.append(line[:-1])
        elif i%3==1:
            sequences.append(line[:-1])

<h1> Easy Version 

In [ ]:
#easy version
new_sequences = []

tik = time.time()
for target, seq in zip(structures, sequences):
    n = len(target)
    model = ir.Model(n, 4)
    model.add_constraints(rna.BPComp(i,j) for (i,j) in rna.parse(target))
    model.add_functions([rna.GCCont(i) for i in range(n)], 'gc')
    
    base_count = Counter(seq)
    GC = (base_count["G"]+base_count["C"]) / len(seq)
    model.set_feature_weight(GC, 'gc')
    
    sampler = ir.Sampler(model)
    sample = sampler.sample()
    new_seq = rna.ass_to_seq(sample)
    new_sequences.append(new_seq)

tok = time.time()
calc_time_easy = (tok-tik)/60
print(f"search took {calc_time_easy:.2f} minutes for {len(sequences)} sequences")

<h1> Frequency table

In [ ]:
def single_target_design_model(target):
    n, bps = len(target), rna.parse(target)
    model = ir.Model(n, 4)
    model.add_constraints(rna.BPComp(i, j) for (i, j) in bps)
    model.add_functions([rna.GCCont(i) for i in range(n)], 'gc')
    model.add_functions([rna.BPEnergy(i, j, (i-1, j+1) not in bps) for (i,j) in bps], 'energy')
    #model.set_feature_weight(-1.5, 'energy')
    return model


def target_frequency(sequence, target):
    fc = RNA.fold_compound(sequence)
    fc.pf()
    return fc.pr_structure(target)

In [ ]:
new_sequences = []

tik = time.time()
#for target, seq in zip(structures, sequences):
for j, target in enumerate(structures):
    n = len(target)
    sampler = ir.Sampler(single_target_design_model(target))
    #base_count = Counter(seq)
    #GC = (base_count["G"]+base_count["C"]) / len(seq)
    #model.set_feature_weight(GC, 'gc')
    
    best = 0
    best_seq = None
    for i in range(100):
        new_seq = rna.ass_to_seq(sampler.targeted_sample())
        freq = target_frequency(new_seq,target)
        if freq > best:
            best = freq
            best_seq = new_seq
            #print(f"{i} {best_seq} {freq:.6f}")
    if best_seq:
        new_sequences.append(best_seq)
    else:
        new_sequences.append(new_seq)
    if j%10 == 0:
        print(f"{j} of {len(structures)} structures done")
        

tok = time.time()
calc_time_easy = (tok-tik)/60
print(f"search took {calc_time_easy:.2f} minutes for {len(sequences)} sequences")

<h1> Sampling with constraint generation

In [ ]:
new_sequences = []

tik = time.time()

for j, target in enumerate(structures):
    n = len(target)
    bps = rna.parse(target)
    def cg_design_iteration():
        model = single_target_design_model(target)
        model.add_constraints(rna.NotBPComp(i, j) for (i, j) in dbps)
        sampler = ir.Sampler(model, lazy=True)
        if sampler.treewidth() > 10 or not sampler.is_consistent():
            return "Not found"
        ctr = Counter()
        found, sol = False, None
        for i in range(50):
            seq = rna.ass_to_seq(sampler.targeted_sample())
            fc = RNA.fold_compound(seq)
            mfe, mfe_e = fc.mfe()
            if fc.eval_structure(target) == mfe_e:
                sol = seq
            ctr.update(rna.parse(mfe))
        ndbps = [x[0] for x in ctr.most_common() if x[0] not in bps]
        dbps.extend(ndbps[:2])
        return sol
    dbps, seq = [], None
    while seq is None:
        seq = cg_design_iteration()
    if seq:
        new_sequences.append(seq)
    if j%10 == 0:
        print(f"{j} of {len(structures)} structures done")

tok = time.time()
calc_time_easy = (tok-tik)/60
print(f"search took {calc_time_easy:.2f} minutes for {len(sequences)} sequences")